In [1]:
import requests
import os
from urllib.parse import urlparse, parse_qs
import json
import numpy as np
import soundfile as sf
import note_seq
import librosa

C:\Users\ok\anaconda3\envs\Environment_1\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
API_KEY = "clzg6zq880004jy0cykcuqehm"
BASE_URL = 'https://api.musicfy.lol/v1'

In [3]:


# Function to generate music from text
def generate_music_from_text(text_description, genre):
    url = f"{BASE_URL}/generate-music"
    instruments = []
    all_midis = []

    pop = ("piano", "violin", "guitar", "synthesizer", "drums", "flute", "vocals")
    jazz = ("saxophone", "trumpet", "trombone", "piano", "bass", "drums", "guitar")
    hip_hop = ("drum machine", "keyboard", "piano", "turntable", "sampler", "bass", "guitar")
    gospel = ("piano", "Hammond organ", "tambourines", "drums", "bass guitar", "keyboard", "electric guitar")
    
    
    if genre == "pop":
        instruments = pop
    elif genre == "jazz":
        instruments = jazz
    elif genre == "hip-hop":
        instruments = hip_hop
    elif genre == "gospel":
        instruments = gospel
    else:
        print("Sorry, selected genre not configured yet!!")
      

    headers = {
        'Authorization': f'Bearer {API_KEY}',
        'Content-Type': 'application/json'
    }

    for item in instruments:
        data = {
            'prompt': item,
            'text': text_description,
            'genre': genre
        }

        response = requests.request("POST",url, headers=headers, json=data).text

        #Using json formatting to extract the html link from the response of the request
        data = json.loads(response)
        file_url = data[0]['file_url']
        
        all_midis.append(file_url)
        
    #print(response.text)
    return all_midis


In [4]:

def download_wav_files(links):
    wav_files = []
    for index, link in enumerate(links):
        try:
            response = requests.get(link)
            response.raise_for_status()  # Check if the request was successful

            parsed_url = urlparse(link)
            file_name = os.path.basename(parsed_url.path)
            file_extension = os.path.splitext(file_name)[1]
            
            if not file_extension:
                file_extension = '.wav'
                
            filename = f'wave_{index}{file_extension}'

            with open(filename, 'wb') as midi_file:
                midi_file.write(response.content)
            wav_files.append(filename)
            
        except requests.exceptions.RequestException as e:
            print(f"Failed to download {link}: {e}")
    return wav_files


In [5]:
"""
import tensorflow as tf

import magenta.music as mm
from magenta.models.music_vae import TrainedModel
from magenta.models.music_vae.configs import CONFIG_MAP
from magenta.models.onsets_frames_transcription import configs, onsets_frames_transcription_transcribe, data

from note_seq.protobuf import music_pb2
from note_seq.sequences_lib import concatenate_sequences
from pydub import AudioSegment
"""

'\nimport tensorflow as tf\n\nimport magenta.music as mm\nfrom magenta.models.music_vae import TrainedModel\nfrom magenta.models.music_vae.configs import CONFIG_MAP\nfrom magenta.models.onsets_frames_transcription import configs, onsets_frames_transcription_transcribe, data\n\nfrom note_seq.protobuf import music_pb2\nfrom note_seq.sequences_lib import concatenate_sequences\nfrom pydub import AudioSegment\n'

In [15]:

def blend_wav_files(wav_paths, output_path):
    valid_wavs = []
    
    for path in wav_paths:
        try:
            # Try loading the WAV file to ensure it is valid
            wav, sr = librosa.load(path, sr=None)
            valid_wavs.append((wav, sr))
        except Exception as e:
            print(f"Failed to load {path}: {e}")
    
    if not valid_wavs:
        print("No valid WAV files to blend.")
        return
    
    # Find the minimum length to avoid issues with different lengths
    min_length = min([len(wav[0]) for wav in valid_wavs])
    
    # Truncate all wav files to the same length
    wavs = [wav[0][:min_length] for wav in valid_wavs]
    
    # Stack and average the wav files to blend them
    blended_wav = np.mean(wavs, axis=0)

    # Delete the content of the output file if it exists
    if os.path.exists(output_path):
        print(f"Output file {output_path} already exists and will be replaced.")
        os.remove(output_path)
    else:
        print(f"Output file {output_path} does not exist. Creating it now.")
    
    # Save the blended wav file
    sf.write(output_path, blended_wav, valid_wavs[0][1])
    print(f"Blended WAV file saved as {output_path}")


In [18]:
# Example usage
if __name__ == "__main__":
    text_description = input("Describe the type of beat you want to create: ").lower()
    genre = input("Select a genre: ").lower()
    
    # Output file name
    output_file = 'final_beat.wav'
    try:
        result = generate_music_from_text(text_description, genre)
        print("Music generated successfully!")

        #for i, items in enumerate(result):
            #print(f'item {i}, download: {items}')

        downloaded_files = download_wav_files(result)
        print(f"Downloaded files: {downloaded_files}")

        final_output = blend_wav_files(downloaded_files, output_file)
    except Exception as e:
        print(e)


Describe the type of beat you want to create:  jazz
Select a genre:  jazz


Output file final_beat.wav already exists and will be replaced.
Blended WAV file saved as final_beat.wav


In [22]:
"""#Converting wav files to midi files
CHECKPOINT_PATH = "../maestro_checkpoint/model.ckpt"

def load_model():
    model_config = configs.CONFIG_MAP['onsets_frames']
    hparams = model_config.hparams
    hparams.use_cudnn = tf.test.is_gpu_available()
    return onsets_frames_transcription_transcribe.run(model_config, CHECKPOINT_PATH)


def transcribe_wav_to_midi(wav_path):
    model = load_model()
    midi_files =[]
    for index, item in enumerate(wav_path):
        midi_path = f'midi_{index}.mid'
        
        samples = note_seq.audio_io.wav_data_to_samples(item, sample_rate=16000)
        input_tensors = data.classify_audio_to_input_tensors(samples, hparams)
        predictions = model.predict(input_tensors, input_tensors)
        sequence = model.predict_to_sequence(predictions, samples)
        note_seq.note_sequence_to_midi_file(sequence, midi_path)
        midi_files.append(midi_path)
        
    return midi_files

#wav_path = downloaded_files
#transcribe_wav_to_midi(wav_path)"""

'#Converting wav files to midi files\nCHECKPOINT_PATH = "../maestro_checkpoint/model.ckpt"\n\ndef load_model():\n    model_config = configs.CONFIG_MAP[\'onsets_frames\']\n    hparams = model_config.hparams\n    hparams.use_cudnn = tf.test.is_gpu_available()\n    return onsets_frames_transcription_transcribe.run(model_config, CHECKPOINT_PATH)\n\n\ndef transcribe_wav_to_midi(wav_path):\n    model = load_model()\n    midi_files =[]\n    for index, item in enumerate(wav_path):\n        midi_path = f\'midi_{index}.mid\'\n        \n        samples = note_seq.audio_io.wav_data_to_samples(item, sample_rate=16000)\n        input_tensors = data.classify_audio_to_input_tensors(samples, hparams)\n        predictions = model.predict(input_tensors, input_tensors)\n        sequence = model.predict_to_sequence(predictions, samples)\n        note_seq.note_sequence_to_midi_file(sequence, midi_path)\n        midi_files.append(midi_path)\n        \n    return midi_files\n\n#wav_path = downloaded_files\n#t

In [24]:
"""
def merge_midi_files(midi_files, output_file):
    #Merge multiple MIDI files into a single MIDI file.
    # Initialize an empty NoteSequence
    combined_sequence = music_pb2.NoteSequence()
    
    for midi_file in midi_files:
        # Read the MIDI file into a NoteSequence
        sequence = mm.midi_file_to_note_sequence(midi_file)
        
        # Append the notes to the combined sequence
        combined_sequence.notes.extend(sequence.notes)
        
        # Adjust the total time of the combined sequence
        combined_sequence.total_time = max(combined_sequence.total_time, sequence.total_time)
    
    # Convert the NoteSequence back to a MIDI file
    mm.sequence_proto_to_midi_file(combined_sequence, output_file)
    
    print(f"Merged MIDI file saved as {output_file}")

"""

'\ndef merge_midi_files(midi_files, output_file):\n    #Merge multiple MIDI files into a single MIDI file.\n    # Initialize an empty NoteSequence\n    combined_sequence = music_pb2.NoteSequence()\n    \n    for midi_file in midi_files:\n        # Read the MIDI file into a NoteSequence\n        sequence = mm.midi_file_to_note_sequence(midi_file)\n        \n        # Append the notes to the combined sequence\n        combined_sequence.notes.extend(sequence.notes)\n        \n        # Adjust the total time of the combined sequence\n        combined_sequence.total_time = max(combined_sequence.total_time, sequence.total_time)\n    \n    # Convert the NoteSequence back to a MIDI file\n    mm.sequence_proto_to_midi_file(combined_sequence, output_file)\n    \n    print(f"Merged MIDI file saved as {output_file}")\n\n'